In [1]:
from models.user import User
from models.scheduleEntry import ScheduleEntry
from models.mapbox import MapboxOptimizer
import firebase
import random
from datetime import datetime, time, timedelta
import folium

firebase.delete_all_collections()

In [2]:
def display_route(route, waypoints=None):
    route_coords = [(point['lat'], point['lng']) for point in route['geometry']]

    # Center map on the first point
    m = folium.Map(location=route_coords[0], zoom_start=15)

    # Add the route as a PolyLine
    folium.PolyLine(route_coords, color="blue", weight=5, opacity=0.8).add_to(m)

    if waypoints is not None:
        new_waypoints = [waypoints[i] for i in route["waypoint_indices"]]
        for waypoint in new_waypoints:
            folium.Marker(waypoint).add_to(m)
    else:
        folium.Marker(route_coords[0], tooltip="Start").add_to(m)
        folium.Marker(route_coords[-1], tooltip="End").add_to(m)
    # Display the map
    return m

In [3]:
def generate_random_date():
    today = datetime.now().date()
    one_week_later = today + timedelta(days=7)
    delta_days = (one_week_later - today).days
    random_days = random.randint(0, delta_days)
    return today + timedelta(days=random_days)

def generate_random_time(start_hour=6, end_hour=19):
    hour = random.randint(start_hour, end_hour-1)
    minute = random.choice([0, 15, 30, 45])
    return time(hour, minute)

In [4]:
users = []
for i in range(5):
    users.append(User(save_object=True))

In [5]:
scheduleEntries = []
for i in range(20):
    user: User = random.choice(users)
    role = random.choice(["driver", "rider"])
    date = generate_random_date()
    start_time = generate_random_time()
    arrival_delta = timedelta(minutes=random.randint(15, 90))
    arrival_datetime = datetime.combine(date, start_time) + arrival_delta
    arrival_time = arrival_datetime.time()
    max_delay = random.randint(10, 30)
    direction = "work" if start_time < time(12, 0) else "home"
    scheduleEntries.append(ScheduleEntry(user.id, role=role, date=date, start_time=start_time, arrival_time=arrival_time, max_delay=max_delay, save_object=True, direction=direction))

In [12]:
rides = firebase.get_all_rides()
#for ride in rides:
#    driver = firebase.get_user(ride.user_id)
    #print(f"Ride driver: {driver.firstName} {driver.lastName}")
print(len(rides))

7


In [14]:
firebase_users = firebase.get_all_users();
for user in firebase_users:
    schedules = firebase.get_schedule_for_user(user_id=user.id)
    print(f"Found {len(schedules)} schedule entries for {user.firstName} {user.lastName}")
    for schedule in schedules:
        print(schedule.start_time, schedule.max_delay, schedule.role)
    print("")

Found 4 schedule entries for Hiba Ayedi
11:45:00 11 driver
08:15:00 21 rider
17:00:00 17 driver
16:15:00 20 rider

Found 4 schedule entries for Kaisoun Mzali
17:00:00 17 rider
12:15:00 18 rider
16:30:00 30 driver
11:00:00 28 rider

Found 5 schedule entries for Souad Ben Romdhane
15:00:00 28 rider
17:00:00 22 driver
15:30:00 12 driver
11:30:00 21 rider
07:00:00 22 driver

Found 5 schedule entries for Hiba Sayyadi
16:00:00 12 driver
09:45:00 29 rider
07:00:00 29 rider
14:30:00 10 rider
18:15:00 28 rider

Found 2 schedule entries for Leila Nafzaoui
13:00:00 27 rider
15:00:00 16 rider



In [10]:
optimizer = MapboxOptimizer()

In [11]:
db = firebase.db
stihl_users = [User.from_dict(doc.to_dict()) for doc in db.collection('users').where('company', '==', 'STIHL').stream()]
for user in stihl_users:
    print(user.firstName)

Hiba
Hiba
Leila
